<h1 align="center">Python performance exercises</h1>

## Python best practices exercises

### Exercise 1

considering the following function for concatenating list strings with delimiter.

In [4]:
!pip install line_profiler

In [19]:
def ft_concatenate(l_strings, d):
    """concatenate list of strings into one string separated by delimiter"""
    res = l_strings[0]
    for e in l_strings[1:]:
        res = res + d + e
    return res

In [83]:
from random import choice
from string import ascii_lowercase, digits

chars = ascii_lowercase + digits

lst = [''.join(choice(chars) for _ in range(2)) for _ in range(10000)]


In [5]:
%load_ext line_profiler

In [28]:
%timeit ft_concatenate(lst,",")

9.29 ms ± 861 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [36]:
%lprun -f ft_concatenate ft_concatenate(lst,",")

As we notice the operation concatination represent the bottlenecks because we read in each step all chars already concataneted so we read first string n time and the second n-1...
the total is nl1+(n-1)l2....ln (l represent the len of the string)

- profile the function and identify the bottlenecks.
- improve speed up of the function
*Hint: you may need to look to the string functions in python documentation*

In [33]:
# write your code here
def ft_concatenate_fast(l_strings, d):
    """concatenate list of strings into one string separated by delimiter"""
    return d.join(l_strings)

In [38]:
%lprun -f ft_concatenate_fast ft_concatenate_fast(lst,",")

### Exercise 2

In this exercise you will solve the following problem using two methods bruteforce method, and fast method.

**Problem:** You are given a list of n integers, and your task is to calculate the number of distinct values in the list.

**Example**
- Input:
5
2 3 2 2 3

- Output:
3

**Implement the following methods:**

1. **bruteforce method:** create an empty list and start adding items for the given list without adding the previous item add, at the end the result list will contain unique values, print lenght of the list and you are done. 
2. **fast method** think of using Set data structure.

- time the two methods, what do you think?

In [8]:
# bruteforce method
def get_distinct_bf(l):
    dis=[]
    for i in l:
        if i not in dis:
            dis.append(i)
    return len(dis)

In [9]:
# fast method
def get_distinct_fast(l):
    return len(set(l))

In [41]:
# Create a random list of numbers for testing
import numpy as np 
l=np.random.choice((1,3000),1000)

# time the two methods
%timeit get_distinct_fast(l)


71.4 µs ± 5.97 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [42]:
%timeit get_distinct_bf(l)

95.2 µs ± 13.5 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## Cython exercises

### Exercise 1

1. load the cython extension.

In [64]:
%load_ext cython

The cython extension is already loaded. To reload it, use:
  %reload_ext cython


2. Considering the following polynomial function:

In [47]:
def poly(a,b):
    return 10.5 * a + 3 * (b**2)

- Create an equivalent Cython function of `poly` with name `poly_cy`.

In [69]:
%%cython 
def poly_cy(double a, double b):
    return 10.5 * a + 3 * (b**2)

3. time the performance of Python and Cython version of the function, what is the factor of speed up between the two verions.

In [70]:
# write your code here
%timeit poly(20, 2)
%timeit poly_cy(20, 2)

429 ns ± 115 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
180 ns ± 19.3 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


4. Now let's work on another example using loop.
    - rewrite the same function below fib that calculates the fibonacci sequence using cython, but now try to add type for the variables used inside it, add a prefix `_cy` to your new cython function.

In [60]:
def fib(n):
    a, b = 1, 1
    for i in range(n):
        a, b = a + b, a

    return a

In [72]:
%%cython
def fib_cy(int n):
    cdef int i, a, b
    a, b = 1, 1
    for i in range(n):
        a, b = a + b, a
    return a

- time the two function for fibonacci series, with n = 20, what is the factor of speed now, What do you think?

In [73]:
# write your code here
%timeit fib(20)
%timeit fib_cy(20)

1.64 µs ± 154 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
80 ns ± 15.5 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


We acheive *20 speed using cython

5. Recursive functions are functions that call themselves during their execution. Another interesting property of the Fibonacci sequence is that it can be written as a recursive function. That’s because each item depends on the values of other items (namely item n-1 and item n-2)

- Rewrite the fib function using recursion. Is it faster than the non-recursive version? Does Cythonizing it give even more of an advantage? 

In [81]:
%%cython
def fib_rec(int n):
    if n <= 1:
        return n
    else:
        return fib_rec(n-1) + fib_rec(n-2)

In [82]:
%timeit fib_rec(20)

566 µs ± 25.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


We remarque despite using cython with recursive fibonacci is less speed in comparaison with the other method because we are calling F(18) twice, F(17) 3 times, F(16) 5 times, F(15) 7 times.. and so on

### Exercise 2

- Monte Carlo methods are a broad class of computational algorithms that rely on repeated random sampling to obtain numerical results. 
- One of the basic examples of getting started with the Monte Carlo algorithm is the estimation of Pi.

**Estimation of Pi**

- The idea is to simulate random (x, y) points in a 2-D plane with domain as a square of side 1 unit. 
- Imagine a circle inside the same domain with same diameter and inscribed into the square. 
- We then calculate the ratio of number points that lied inside the circle and total number of generated points. 
- Refer to the image below:

![demo](../data/MonteCarloPlot.png)

We know that area of the square is 1 unit sq while that of circle is $\pi \ast  (\frac{1}{2})^{2} = \frac{\pi}{4}$. Now for a very large number of generated points,

![demo](../data/MonteCarloCalc.png)


## The Algorithm

1. Initialize cile_points, square_points and interval to 0.
2. Generate random point x.
3. Generate random point y.
4. Calculate d = x*x + y*y.
5. If d <= 1, increment circle_points.
6. Increment square_points.
7. Increment interval.
8. If increment < NO_OF_ITERATIONS, repeat from 2.
9. Calculate pi = 4*(circle_points/square_points).
10. Terminate.

**Your mission:** time the function `monte_carlo_pi`, identify the bottlenecks and create a new version using cython functionality to speed up monte carlo simulation for PI, use 100,000 points and compare the speed up factor between python and cython, considering the following optimizations:
- add type for variables used.
- add type for the function
- use c rand function instead of python rand function.
 
*Hint: you can import function from C libraries using the following approach `from libc.<name of c library> cimport <library function name>`, replace the holders `<>` with the right identities for the current problem*

In [84]:
import random
def monte_carlo_pi(nsamples):
    pi = 0.
    
   # Implement your code here
    cile_points=0
    square_points=0 
    for i in range(nsamples):
        x = random.random()
        y = random.random()
        d = x**2 + y**2
        if d<=1:
            cile_points += 1
        square_points += 1
    pi=4*(cile_points/square_points)
    return pi

In [118]:
%%cython
from libc.stdlib cimport rand
cdef extern from "limits.h":
    int INT_MAX

cdef float randnum = rand() / float(INT_MAX)
print(randnum)

In [137]:
%%cython
import random
from libc.stdlib cimport rand
def monte_carlo_pi_cy(int nsamples):
    cdef float pi = 0
    
   # Implement your code here
    cdef int cile_points=0
    cdef int square_points=0 
    cdef float x, y, d
    cdef extern from "limits.h":
        int INT_MAX
    for i in range(nsamples):
        x = rand() / float(INT_MAX)
        y = rand() / float(INT_MAX)
        #x = random.random()
        #y = random.random()
        d = x**2 + y**2
        if d<=1:
            cile_points += 1
        square_points += 1
    pi=4*(cile_points/square_points)
    return pi

In [129]:
print(monte_carlo_pi(100000))
print(monte_carlo_pi_cy(100000))

3.13816
3.144399881362915


In [130]:
%timeit monte_carlo_pi(100000)
%timeit monte_carlo_pi_cy(100000)

72.2 ms ± 9.74 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
10.6 ms ± 1.08 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Numba exercises

### Exercise 1

Previously we considered how to approximateby Monte Carlo.

- Use the same idea here, but make the code efficient using Numba.
- Compare speed with and without Numba when the sample size is large.

In [160]:
import numba
print(numba.__version__)

0.54.1


In [179]:
%%cython
from libc.stdlib cimport rand
import numba
from numba import jit
@jit(nopython=True)
def monte_carlo_numba(int nsamples):
    cdef float pi = 0
   # Implement your code here
    cdef int cile_points=0
    cdef int square_points=0 
    cdef float x, y, d
    cdef extern from "limits.h":
        int INT_MAX
    for i in range(nsamples):
        x = rand() / float(INT_MAX)
        y = rand() / float(INT_MAX)
        #x = random.random()
        #y = random.random()
        d = x**2 + y**2
        if d<=1:
            cile_points += 1
        square_points += 1
    pi=4*(cile_points/square_points)
    return pi


In [180]:
%timeit monte_carlo_pi(100000)
%timeit monte_carlo_pi_cy(100000)
%timeit monte_carlo_numba(100000)


76.6 ms ± 16.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
11.8 ms ± 1.82 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


NameError: name 'monte_carlo_numba' is not defined